In [1]:
import sys
%matplotlib inline
import numpy as np
import os
import matplotlib.pyplot
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as mticker
import matplotlib.dates as mdates
import datetime
from tqdm import tqdm
import GPy
from collections import defaultdict
from pathlib import Path
import seaborn as sns
import scipy.stats as stats
from matplotlib.colors import ListedColormap
import warnings
import time
from itertools import product
from joblib import Parallel, delayed
from operator import itemgetter

mpl.rcParams['legend.frameon'] = False
mpl.rcParams['figure.autolayout'] = True
# mpl.rcParams['figure.dpi'] = 300
# mpl.rcParams['axes.spines.right'] = False
# mpl.rcParams['axes.spines.top'] = False


plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"]})

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
})


def utkarshGrid():
    plt.minorticks_on()
    plt.grid(color='grey',
             which='minor',
             linestyle=":",
             linewidth='0.1',
             )
    plt.grid(color='black',
             which='major',
             linestyle=":",
             linewidth='0.1',
             )

In [2]:
path = os.path.dirname(os.getcwd())
os.chdir(path)
from Emulator.Classes.AllData import AllData
from Emulator.Classes.LightCurve import utkarshGrid, LightCurve
from Emulator.Classes.GP import GP
from Emulator.Classes.GP2D import GP2D
from Emulator.Classes.GP5D import GP5D
os.chdir(os.getcwd() + "/Emulator")
os.getcwd()

'/Users/utkarsh/PycharmProjects/KilonovaeLightCurves/Emulator'

In [3]:
curr_wv = np.arange(100, 3600, 10)
set_n_comp = 25
set_skip_factor = None

In [4]:
# gp = GP5D("Classes/reference.csv")
# gp.skip_factor = set_skip_factor
# gp.set_wv_range(curr_wv)
# gp.n_comp = set_n_comp
# gp.num_pca_components = (4,4)
# time_shape = None
# gp.setXY()
# # gp.save_pca_components(skip_factor = None)
# gp.save_trained_data()
# gp.ComputeDifferenceFlux()
# gp.plot_difference_histogram(binning = 150)
# plt.title("Error in only PCA Components (PCA Flux - Truth)/TRUTH")
# plt.xlabel("Units Log Flux (Error)")
# plt.ylabel("Count")
# plt.xlim(-1, 1)
# plt.show()

In [ ]:
gp = GP5D("Classes/reference.csv")
gp.set_wv_range(curr_wv)
gp.n_comp = set_n_comp
time_shape = None
gp.num_pca_components = (10,10)
# gp.save_training_data(time_trauncation = time_shape) # Which wv do you want to extract.  
gp.save_pca_components(skip_factor = set_skip_factor)
gp.setXY()
fitting_kernel = GPy.kern.RBF(input_dim=4, variance = 10, lengthscale=10, ARD = True)
decay_kernel = GPy.kern.Linear(4 , ARD=True)
gp.kernel = fitting_kernel * decay_kernel
gp.model = GPy.models.GPRegression(gp.X,gp.Y,gp.kernel)
t0 = time.time()
print(f"[STATUS] Optimizing...")
gp.model.optimize_restarts(parallel = True, robust = True, num_restarts = 20)
print(f"[STATUS] {round(time.time() - t0, 2)}s taken to optimize.")
# gp.model.optimize(messages = True)
# display(gp.model.rbf.lengthscale)
display(gp.model.mul.rbf.lengthscale)
t1 = time.time()
gp.model_predict()
print(f"[STATUS] {round(time.time() - t1, 2)}s taken to predict.")
t2 = time.time()
gp.save_trained_data()
print(f"[STATUS] {round(time.time() - t2, 2)}s taken to save data.")
# gp.ComputeDifferenceFlux()
# gp.plot_difference_histogram(binning = 150)
# plt.xlim(-1,1)
# plt.show()

[STATUS] Setting X, Y components for 5D Model.
[STATUS] Optimizing...
Optimization restart 1/20, f = 205456.40226213407
Optimization restart 2/20, f = 203117.22728400785
Optimization restart 3/20, f = 207420.19900313995
Optimization restart 4/20, f = 236290.72938201507
Optimization restart 5/20, f = 203106.70512859686
Optimization restart 6/20, f = 203119.16996816354
Optimization restart 7/20, f = 205889.83116504658
Optimization restart 8/20, f = 204919.66134819214
Optimization restart 9/20, f = 204217.37767821248
Optimization restart 10/20, f = 202497.23484769286
Optimization restart 11/20, f = 224008.15962655755
Optimization restart 12/20, f = 206681.70853942668
Optimization restart 13/20, f = 204247.70630686224
Optimization restart 14/20, f = 211927.46124105365
Optimization restart 15/20, f = 208420.59892375008
Optimization restart 16/20, f = 216800.84692616804
Optimization restart 17/20, f = 219771.16618331498
Optimization restart 18/20, f = 214563.66558435804
Optimization restart 

index,GP_regression.mul.rbf.lengthscale,constraints,priors
[0],0.12862882,+ve,
[1],0.10035217,+ve,
[2],0.11685431,+ve,
[3],108.28621897,+ve,


[STATUS] Predicting X and Y with trained emulator.
[STATUS] 2.11s taken to predict.


In [ ]:
# WHAT WAVELENGTH AND TIME WOULD YOU LIKE? 
wv_desired = 250 #nm
time_desired = 5 #day

mejdyn = 0.001
mejwind =  0.01
phi = 45
iobs = 4

t = np.arange(0, 20, 20/100)
untrained = np.load(f"pca/mejdyn{mejdyn}_mejwind{mejwind}_phi{phi}_iobs{iobs}.npy")
trained = np.load(f"pcaTrained/mejdyn{mejdyn}_mejwind{mejwind}_phi{phi}_iobs{iobs}.npy")
trainedUpper = np.load(f"pcaTrainedUpper/mejdyn{mejdyn}_mejwind{mejwind}_phi{phi}_iobs{iobs}.npy")
trainedLower = np.load(f"pcaTrainedLower/mejdyn{mejdyn}_mejwind{mejwind}_phi{phi}_iobs{iobs}.npy")
wv_index = (np.abs(gp.wv_range - wv_desired)).argmin() # Plot arbitary wavelength. 
time_index = (np.abs(t - time_desired)).argmin() # Plot arbitary wavelength. 

In [ ]:
import sncosmo
filters = ["sdss::u", "sdss::g", "sdss::r", "sdss::i", "sdss::z",
           "swope2::y", "swope2::J", "swope2::H"]

colors = sns.color_palette("coolwarm_r",len(filters))[::-1]

plt.figure(dpi = 300, figsize = (6,3))
for i in range(len(filters)):
    source = sncosmo.TimeSeriesSource(t, gp.wv_range*10, 10 ** trained)
    source2 = sncosmo.TimeSeriesSource(t, gp.wv_range*10, 10 ** untrained)
    m = source.bandmag(filters[i], "ab", t)
    m2 = source2.bandmag(filters[i], "ab", t)
    plt.plot(t, m, label = f"{filters[i][-1]}", color = colors[i])
    plt.plot(t, m2, linestyle = "dotted", alpha = 0.3, color = colors[i])

plt.legend()
utkarshGrid()
plt.gca().invert_yaxis()
plt.show()

In [ ]:

print(f"=== Flux Estimation === \nmejdyn: {mejdyn}\nmejwind: {mejwind}\
\nphi: {phi}\nviewing_angle: {iobs}\nwavelength: {gp.wv_range[wv_index]}nm\
\ntime: {round(t[time_index],2)} days\n\nLOG FLUX: {round(trained[time_index,wv_index], 5)}")


plt.figure(dpi = 300, figsize = (6,3))
plt.title(f"Wavelength = {gp.wv_range[wv_index]}nm")
plt.plot(t, untrained[:,wv_index], label = "Training Data", color = "purple")
plt.plot(t, trainedUpper[:,wv_index], alpha = 0.3, color = "lightblue", label = r"1$\sigma$")
plt.plot(t, trainedLower[:,wv_index], alpha = 0.3, color = "lightblue")
plt.plot(t, trained[:,wv_index], label = "Trained Emulator + PCA", linestyle = "dashed", color = "dodgerblue")
plt.legend()
plt.xlabel("Time (s)")
plt.ylabel("Log Flux (Magnitude)")
utkarshGrid()
plt.show()

plt.figure(dpi = 300, figsize = (6,3))
plt.title(f"Time = {round(t[time_index],3)} Days")
plt.plot(gp.wv_range, untrained[time_index,:], label = "Training Data", color = "purple")
plt.plot(gp.wv_range, trained[time_index,:], label = "Trained Emulator + PCA", 
         linestyle = "dashed", color = "dodgerblue")
plt.plot(gp.wv_range, trainedUpper[time_index,:], alpha = 0.3, color = "lightblue" , label = r"1$\sigma$ Error (UNFINISHED)")
plt.plot(gp.wv_range, trainedLower[time_index,:], alpha = 0.3, color = "lightblue")
plt.legend()
plt.xlabel("Wavelength (nm)")
plt.ylabel("Log Flux (Magnitude)")
utkarshGrid()
plt.show()

In [ ]:
# gp = GP5D("Classes/reference.csv")
# gp.set_wv_range(curr_wv)
# gp.n_comp = set_n_comp
# # gp.num_pca_components = (4,4)
# # gp.save_training_data(time_trauncation = None) # Which wv do you want to extract.  
# gp.save_pca_components(skip_factor = set_skip_factor)
# gp.setXY()
# fitting_kernel = GPy.kern.RBF(input_dim=4, variance = 10, lengthscale=10, ARD = True)
# decay_kernel = GPy.kern.Linear(4,ARD=True)
# gp.kernel = fitting_kernel * decay_kernel
# gp.LOOCV_PCA()
# gp.plot_loocv_histogram() # THIS IS FOR PCA COMPONENTS, NOT TRAINING DATA

In [ ]:
# gp.save_trained_data()
# gp.ComputeDifferenceFlux()
# gp.plot_difference_histogram(binning = 100)
# plt.xlim(-1, 1)